# Step Flow

> Multi-step wizard pattern with state management, navigation, and route generation

In [ ]:
#| default_exp patterns.step_flow

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from typing import Dict, Any, Optional, Callable, List
from dataclasses import dataclass, field
from fasthtml.common import *
from fastcore.basics import patch

from cjm_fasthtml_interactions.core.context import InteractionContext
from cjm_fasthtml_interactions.core.html_ids import InteractionHtmlIds
from cjm_fasthtml_interactions.core.state_store import WorkflowStateStore, InMemoryWorkflowStateStore
from cjm_fasthtml_daisyui.components.actions.button import btn, btn_colors, btn_styles
from cjm_fasthtml_daisyui.components.navigation.steps import steps, step, step_colors
from cjm_fasthtml_tailwind.utilities.flexbox_and_grid import flex_display, gap, justify
from cjm_fasthtml_tailwind.utilities.spacing import m, p
from cjm_fasthtml_tailwind.core.base import combine_classes

## Step Definition

The `Step` class defines a single step in a multi-step workflow. Each step has:
- Unique identifier
- Title for display
- Render function that generates the UI
- Optional validation function
- Optional data loader for fetching required data
- State keys that this step manages

In [ ]:
#| export
@dataclass
class Step:
    """Definition of a single step in a multi-step workflow."""
    
    id: str  # Unique step identifier (used in URLs)
    title: str  # Display title for the step
    render: Callable[[InteractionContext], Any]  # Function to render step UI
    validate: Optional[Callable[[Dict[str, Any]], bool]] = None  # Validation function
    data_loader: Optional[Callable[[Any], Dict[str, Any]]] = None  # Data loading function
    data_keys: List[str] = field(default_factory=list)  # State keys managed by this step
    can_skip: bool = False  # Whether this step can be skipped
    show_back: bool = True  # Whether to show back button
    show_cancel: bool = True  # Whether to show cancel button
    next_button_text: str = "Continue"  # Text for next/submit button
    
    def is_valid(self, state: Dict[str, Any]  # Current workflow state
                ) -> bool:  # True if step is complete and valid
        """Check if step has valid data in state."""
        if self.validate:
            return self.validate(state)
        # Default: check that all data_keys are present
        return all(key in state for key in self.data_keys)

## StepFlow Class

The `StepFlow` class manages a multi-step workflow. It:
- Tracks current step
- Manages navigation between steps
- Uses a `WorkflowStateStore` for server-side state persistence
- Generates routes automatically
- Provides resumability (returns to last valid step on page load)

In [ ]:
#| export
class StepFlow:
    """Manage multi-step workflows with automatic route generation and state management."""
    
    def __init__(
        self,
        flow_id: str,  # Unique identifier for this workflow
        steps: List[Step],  # List of step definitions
        state_store: Optional[WorkflowStateStore] = None,  # Storage backend (defaults to InMemoryWorkflowStateStore)
        container_id: str = InteractionHtmlIds.STEP_FLOW_CONTAINER,  # HTML ID for content container
        on_complete: Optional[Callable[[Dict[str, Any], Any], Any]] = None,  # Completion handler
        show_progress: bool = False,  # Whether to show progress indicator
        wrap_in_form: bool = True,  # Whether to wrap content + navigation in a form
        debug: bool = False  # Whether to print debug information
    ):
        """Initialize step flow manager."""
        self.flow_id = flow_id
        self.steps = steps
        self.state_store = state_store or InMemoryWorkflowStateStore()
        self.container_id = container_id
        self.on_complete = on_complete
        self.show_progress = show_progress
        self.wrap_in_form = wrap_in_form
        self.debug = debug
        
        # Build step index for quick lookup
        self.step_index = {step.id: idx for idx, step in enumerate(steps)}

## Step Management Methods

In [ ]:
#| export
@patch
def get_step(self:StepFlow, 
             step_id: str  # Step identifier
            ) -> Optional[Step]:  # Step object or None
    """Get step by ID."""
    idx = self.step_index.get(step_id)
    if idx is not None:
        return self.steps[idx]
    return None

In [ ]:
#| export
@patch
def get_step_index(self:StepFlow, 
                   step_id: str  # Step identifier
                  ) -> Optional[int]:  # Step index or None
    """Get step index by ID."""
    return self.step_index.get(step_id)

## Navigation Methods

In [ ]:
#| export
@patch
def get_current_step_id(self:StepFlow, 
                        sess: Any  # FastHTML session object
                       ) -> str:  # Current step ID
    """Get current step ID from state store."""
    current = self.state_store.get_current_step(self.flow_id, sess)
    if current is None:
        current = self.steps[0].id
    return current

In [ ]:
#| export
@patch
def set_current_step(self:StepFlow, 
                     sess: Any,  # FastHTML session object
                     step_id: str  # Step ID to set as current
                    ) -> None:
    """Set current step in state store."""
    self.state_store.set_current_step(self.flow_id, sess, step_id)

In [ ]:
#| export
@patch
def get_next_step_id(self:StepFlow, 
                     current_step_id: str  # Current step ID
                    ) -> Optional[str]:  # Next step ID or None if last step
    """Get the ID of the next step."""
    idx = self.get_step_index(current_step_id)
    if idx is not None and idx < len(self.steps) - 1:
        return self.steps[idx + 1].id
    return None

In [ ]:
#| export
@patch
def get_previous_step_id(self:StepFlow, 
                         current_step_id: str  # Current step ID
                        ) -> Optional[str]:  # Previous step ID or None if first step
    """Get the ID of the previous step."""
    idx = self.get_step_index(current_step_id)
    if idx is not None and idx > 0:
        return self.steps[idx - 1].id
    return None

In [ ]:
#| export
@patch
def is_last_step(self:StepFlow, 
                 step_id: str  # Step ID to check
                ) -> bool:  # True if this is the last step
    """Check if step is the last step."""
    idx = self.get_step_index(step_id)
    return idx == len(self.steps) - 1 if idx is not None else False

In [ ]:
#| export
@patch
def is_first_step(self:StepFlow, 
                  step_id: str  # Step ID to check
                 ) -> bool:  # True if this is the first step
    """Check if step is the first step."""
    idx = self.get_step_index(step_id)
    return idx == 0 if idx is not None else False

## State Management Methods

In [ ]:
#| export
@patch
def get_workflow_state(self:StepFlow, 
                       sess: Any  # FastHTML session object
                      ) -> Dict[str, Any]:  # All workflow state
    """Get all workflow state from state store."""
    return self.state_store.get_state(self.flow_id, sess)

In [ ]:
#| export
@patch
def update_workflow_state(self:StepFlow, 
                          sess: Any,  # FastHTML session object
                          updates: Dict[str, Any]  # State updates
                         ) -> None:
    """Update workflow state with new values."""
    # Button IDs to exclude from state (these are just UI artifacts)
    button_keys = {
        InteractionHtmlIds.STEP_FLOW_NEXT_BTN,
        InteractionHtmlIds.STEP_FLOW_SUBMIT_BTN,
        InteractionHtmlIds.STEP_FLOW_BACK_BTN,
        InteractionHtmlIds.STEP_FLOW_CANCEL_BTN
    }
    # Filter out private keys and button IDs
    filtered_updates = {
        key: value for key, value in updates.items()
        if not key.startswith("__") and key not in button_keys
    }
    if filtered_updates:
        self.state_store.update_state(self.flow_id, sess, filtered_updates)

In [ ]:
#| export
@patch
def clear_workflow(self:StepFlow, 
                   sess: Any  # FastHTML session object
                  ) -> None:
    """Clear all workflow state."""
    self.state_store.clear_state(self.flow_id, sess)

In [ ]:
#| export
@patch
def _summarize_state(self:StepFlow, 
                     state: Dict[str, Any]  # State dictionary to summarize
                    ) -> str:  # Human-readable summary string
    """Create a concise summary of state for debug output."""
    if not state:
        return "{}"
    
    parts = []
    for key, value in state.items():
        if isinstance(value, list):
            parts.append(f"{key}: list[{len(value)}]")
        elif isinstance(value, dict):
            parts.append(f"{key}: dict[{len(value)} keys]")
        elif isinstance(value, str) and len(value) > 50:
            parts.append(f"{key}: str[{len(value)} chars]")
        else:
            parts.append(f"{key}: {repr(value)}")
    
    return "{" + ", ".join(parts) + "}"

## Context and Rendering Methods

In [ ]:
#| export
@patch
def create_context(self:StepFlow, 
                   request: Any,  # FastHTML request object
                   sess: Any,  # FastHTML session object
                   step: Step  # Current step
                  ) -> InteractionContext:  # Interaction context for rendering
    """Create interaction context for a step."""
    # Get workflow state
    state = self.get_workflow_state(sess)
    
    # Load data if step has data loader
    data = {}
    if step.data_loader:
        data = step.data_loader(request)
    
    return InteractionContext(
        state=state,
        request=request,
        session=sess,
        data=data
    )

In [ ]:
#| export
@patch
def render_progress(self:StepFlow, 
                    sess: Any  # FastHTML session object
                   ) -> FT:  # Progress indicator or empty Div
    """Render progress indicator showing all steps."""
    if not self.show_progress:
        return Div()  # Return empty if progress disabled
    
    current_step_id = self.get_current_step_id(sess)
    current_idx = self.get_step_index(current_step_id)
    
    step_items = []
    for idx, step_def in enumerate(self.steps):
        # Completed steps (and current step) get primary color
        if idx <= current_idx:
            cls = combine_classes(step, step_colors.primary)
        else:
            cls = str(step)
        
        step_items.append(Li(step_def.title, cls=cls))
    
    return Ul(
        *step_items, 
        cls=str(steps), 
        id=InteractionHtmlIds.STEP_FLOW_PROGRESS
    )

In [ ]:
#| export
@patch
def render_step_content(self:StepFlow,
                        step_obj: Step,  # Step to render
                        ctx: InteractionContext,  # Interaction context
                        next_route: str,  # Route for next/submit
                        back_route: Optional[str] = None,  # Route for back
                        cancel_route: Optional[str] = None  # Route for cancel
                       ) -> FT:  # Complete step content with optional progress and navigation
    """Render step content with optional progress indicator and navigation."""
    components = []
    
    # Add progress indicator if enabled
    if self.show_progress:
        components.append(
            Div(
                self.render_progress(ctx.session),
                # cls=combine_classes(m.b(6))
            )
        )
    
    # Add step content
    content = step_obj.render(ctx)
    components.append(content)
    
    # Add navigation
    navigation = self.render_navigation(
        step_id=step_obj.id,
        next_route=next_route,
        back_route=back_route,
        cancel_route=cancel_route
    )
    components.append(navigation)
    
    # Wrap in form if needed (for automatic form data submission)
    if self.wrap_in_form:
        return Form(
            *components,
            hx_post=next_route,
            hx_target=InteractionHtmlIds.as_selector(self.container_id),
            hx_swap="innerHTML"
        )
    else:
        return Div(*components)

In [ ]:
#| export
@patch
def render_navigation(self:StepFlow,
                      step_id: str,  # Current step ID
                      next_route: str,  # Route for next/submit action
                      back_route: Optional[str] = None,  # Route for back action
                      cancel_route: Optional[str] = None,  # Route for cancel action
                     ) -> FT:  # Navigation button container
    """Render navigation buttons for a step."""
    step_obj = self.get_step(step_id)
    if not step_obj:
        return Div()
    
    buttons = []
    
    # Back button
    if step_obj.show_back and back_route and not self.is_first_step(step_id):
        buttons.append(
            Button(
                "← Back",
                hx_get=back_route,
                hx_target=InteractionHtmlIds.as_selector(self.container_id),
                hx_swap="innerHTML",
                type="button",  # Important: prevent form submission
                id=InteractionHtmlIds.STEP_FLOW_BACK_BTN,
                cls=combine_classes(btn, btn_styles.ghost)
            )
        )
    
    # Next/Submit button
    is_last = self.is_last_step(step_id)
    button_text = step_obj.next_button_text
    button_id = InteractionHtmlIds.STEP_FLOW_SUBMIT_BTN if is_last else InteractionHtmlIds.STEP_FLOW_NEXT_BTN
    
    # If wrapped in form, this will submit the form
    # If not wrapped, it will trigger HTMX POST
    button_attrs = {"id": button_id, "cls": combine_classes(btn, btn_colors.primary)}
    if self.wrap_in_form:
        button_attrs["type"] = "submit"
    else:
        button_attrs.update({
            "hx_post": next_route,
            "hx_target": InteractionHtmlIds.as_selector(self.container_id),
            "hx_swap": "innerHTML"
        })
    
    buttons.append(Button(button_text, **button_attrs))
    
    # Cancel button
    if step_obj.show_cancel and cancel_route:
        buttons.append(
            Button(
                "Cancel",
                hx_get=cancel_route,
                hx_target=InteractionHtmlIds.as_selector(self.container_id),
                hx_swap="innerHTML",
                type="button",  # Important: prevent form submission
                id=InteractionHtmlIds.STEP_FLOW_CANCEL_BTN,
                cls=combine_classes(btn, btn_styles.ghost)
            )
        )
    
    return Div(
        *buttons,
        id=InteractionHtmlIds.STEP_FLOW_NAVIGATION,
        cls=combine_classes(
            flex_display, 
            gap(2), 
            justify.end, 
            # m.t(4)
        )
    )

## Route Generation

In [ ]:
#| export
@patch
def create_router(self:StepFlow,
                  prefix: str = ""  # URL prefix for routes (e.g., "/transcription")
                 ) -> APIRouter:  # APIRouter with generated routes
    """Create FastHTML router with generated routes for this flow."""
    router = APIRouter(prefix=prefix)

    # Store reference to flow in router for access in route handlers
    router.step_flow = self

    # Entry point route - start or resume
    @router
    def start(request, sess):
        """Entry point - start workflow or resume from last step."""
        current_step_id = self.get_current_step_id(sess)
        step_obj = self.get_step(current_step_id)

        if not step_obj:
            # Invalid state, restart from beginning
            step_obj = self.steps[0]
            self.set_current_step(sess, step_obj.id)

        ctx = self.create_context(request, sess, step_obj)

        step_content = self.render_step_content(
            step_obj=step_obj,
            ctx=ctx,
            next_route=next_step.to(),
            back_route=back_step.to(),
            cancel_route=reset.to()
        )

        return Div(step_content, id=self.container_id)

    # Next step handler
    @router
    async def next_step(request, sess):
        """Advance to next step."""
        current_step_id = self.get_current_step_id(sess)
        current_step = self.get_step(current_step_id)
        
        if self.debug:
            print(f"DEBUG StepFlow: current_step_id={current_step_id}, is_last={self.is_last_step(current_step_id)}")

        if not current_step:
            return start(request, sess)

        # Get form data if any (properly handle FormData from Starlette)
        try:
            form_data = await request.form()
            # Convert FormData to dict
            form_dict = {key: form_data.get(key) for key in form_data.keys()}

            # Update workflow state with form data
            if form_dict:
                if self.debug:
                    print(f"DEBUG StepFlow: form_dict keys={list(form_dict.keys())}")
                self.update_workflow_state(sess, form_dict)
        except Exception as e:
            if self.debug:
                print(f"DEBUG StepFlow: Exception getting form data: {e}")
            # No form data or error reading it
            pass

        # Validate current step before advancing
        state = self.get_workflow_state(sess)
        if self.debug:
            print(f"DEBUG StepFlow: state={self._summarize_state(state)}")
        is_valid = current_step.is_valid(state)
        if self.debug:
            print(f"DEBUG StepFlow: is_valid={is_valid}")
        
        if not is_valid:
            # Validation failed, re-render current step
            if self.debug:
                print(f"DEBUG StepFlow: Validation failed, re-rendering current step")
            ctx = self.create_context(request, sess, current_step)
            step_content = self.render_step_content(
                step_obj=current_step,
                ctx=ctx,
                next_route=next_step.to(),
                back_route=back_step.to(),
                cancel_route=reset.to()
            )
            return Div(step_content, id=self.container_id)

        # Check if this is the last step
        if self.is_last_step(current_step_id):
            if self.debug:
                print(f"DEBUG StepFlow: This is the last step, calling on_complete")
            # Complete the workflow
            if self.on_complete:
                state = self.get_workflow_state(sess)
                # Check if completion handler is async
                import inspect
                if inspect.iscoroutinefunction(self.on_complete):
                    result = await self.on_complete(state, request)
                    if self.debug:
                        print(f"DEBUG StepFlow: on_complete returned (async)")
                    return result
                else:
                    result = self.on_complete(state, request)
                    if self.debug:
                        print(f"DEBUG StepFlow: on_complete returned (sync)")
                    return result
            else:
                # No completion handler, just show success
                return Div("Workflow completed!", id=self.container_id)

        # Move to next step
        if self.debug:
            print(f"DEBUG StepFlow: Moving to next step")
        next_step_id = self.get_next_step_id(current_step_id)
        if next_step_id:
            self.set_current_step(sess, next_step_id)
            step_obj = self.get_step(next_step_id)

            ctx = self.create_context(request, sess, step_obj)

            step_content = self.render_step_content(
                step_obj=step_obj,
                ctx=ctx,
                next_route=next_step.to(),
                back_route=back_step.to(),
                cancel_route=reset.to()
            )

            return Div(step_content, id=self.container_id)

        return start(request, sess)

    # Back step handler
    @router
    def back_step(request, sess):
        """Go back to previous step."""
        current_step_id = self.get_current_step_id(sess)
        prev_step_id = self.get_previous_step_id(current_step_id)

        if prev_step_id:
            self.set_current_step(sess, prev_step_id)
            prev_step = self.get_step(prev_step_id)

            ctx = self.create_context(request, sess, prev_step)

            step_content = self.render_step_content(
                step_obj=prev_step,
                ctx=ctx,
                next_route=next_step.to(),
                back_route=back_step.to(),
                cancel_route=reset.to()
            )

            return Div(step_content, id=self.container_id)

        return start(request, sess)

    # Reset workflow handler
    @router
    def reset(request, sess):
        """Reset workflow to beginning."""
        self.clear_workflow(sess)
        return start(request, sess)

    return router

## Usage Example

Here's a complete example showing how to create a multi-step workflow. The `StepFlow` class uses `InMemoryWorkflowStateStore` by default, so you don't need to provide a state store for simple use cases.

In [ ]:
# Example: Simple 3-step workflow
from fasthtml.common import Div, H2, P, Form, Input, Label

# Define render functions for each step
def render_step1(ctx: InteractionContext):
    """Render first step - collect name."""
    current_name = ctx.get("name", "")
    return Div(
        H2("Step 1: Enter Your Name"),
        Label("Name:"),
        Input(name="name", value=current_name, required=True)
    )

def render_step2(ctx: InteractionContext):
    """Render second step - collect email."""
    name = ctx.get("name", "User")
    current_email = ctx.get("email", "")
    return Div(
        H2(f"Step 2: Hi {name}, enter your email"),
        Label("Email:"),
        Input(name="email", type="email", value=current_email, required=True)
    )

def render_step3(ctx: InteractionContext):
    """Render third step - confirm."""
    name = ctx.get("name", "")
    email = ctx.get("email", "")
    return Div(
        H2("Step 3: Confirm"),
        P(f"Name: {name}"),
        P(f"Email: {email}")
    )

# Define completion handler
def on_complete(state: Dict[str, Any], request):
    """Handle workflow completion."""
    return Div(
        H2("Success!"),
        P(f"Welcome {state.get('name')}!"),
        P(f"We'll email you at {state.get('email')}")
    )

# Create the step flow - uses InMemoryWorkflowStateStore by default
registration_flow = StepFlow(
    flow_id="registration",
    steps=[
        Step(
            id="name",
            title="Enter Name",
            render=render_step1,
            data_keys=["name"]
        ),
        Step(
            id="email",
            title="Enter Email",
            render=render_step2,
            data_keys=["email"]
        ),
        Step(
            id="confirm",
            title="Confirm",
            render=render_step3,
            next_button_text="Complete Registration"
        )
    ],
    on_complete=on_complete,
    show_progress=True
)

# Generate router
registration_router = registration_flow.create_router(prefix="/register")

# In your FastHTML app, register the router:
# from cjm_fasthtml_app_core.core.routing import register_routes
# register_routes(app, registration_router)
#
# Or directly:
# registration_router.to_app(app)

In [ ]:
# Test step flow navigation
print(f"Flow has {len(registration_flow.steps)} steps")
print(f"First step: {registration_flow.steps[0].id}")
print(f"Last step: {registration_flow.steps[-1].id}")
print(f"Next after 'name': {registration_flow.get_next_step_id('name')}")
print(f"Previous before 'email': {registration_flow.get_previous_step_id('email')}")

Flow has 3 steps
First step: name
Last step: confirm
Next after 'name': email
Previous before 'email': name


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()